## Preambule

In [38]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Paths

In [39]:
path_data = Path("K:/data/DataUpdate_ongoing")
path_figs = Path("K:/Code/EffortSharing/") / "Figures" / "Paper_Robiou2023"

## Read data files

In [40]:
xr_dataread = xr.open_dataset(path_data / "xr_dataread.nc")
xr_combs = xr.open_dataset(path_data / "xr_combinedapproaches.nc")
xr_temps = xr.open_dataset(path_data / "xr_combinedapproaches_temps.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.array(list(np.load(path_data / "all_regions_names.npy")[:-1])+['Earth']) # Only in old xr_dataread version, should be updated when reran
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries", 'World']

In [41]:
#xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
#df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [42]:
df_g = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df_g["Country ISO Code"])
countries_name = np.array(df_g["Name"])
countries_iso = np.array(df_g["Country ISO Code"])
group_cvf = countries_iso[np.array(df_g["CVF (24/10/22)"]) == 1]
group_g20 = countries_iso[np.array(df_g["G20"]) == 1]
group_eu = countries_iso[np.array(df_g["EU"]) == 1]
group_g7 = countries_iso[np.array(df_g["G7"]) == 1]
group_na = countries_iso[np.array(df_g["Northern America"]) == 1]
group_um = countries_iso[np.array(df_g["Umbrella"]) == 1]
group_au = countries_iso[np.array(df_g["Australasia"]) == 1]
group_af = countries_iso[np.array(df_g["African Group"]) == 1]
group_sids = countries_iso[np.array(df_g["SIDS"]) == 1]
group_ldc = countries_iso[np.array(df_g["LDC"]) == 1]
group_eg = countries_iso[np.array(df_g["European Group"]) == 1]
group_world = np.copy(countries_iso)

## Preprocess

In [43]:
ndc_deviations = []
for t_i, t in enumerate([1.6, 2.0]):
    li = []
    for a_i, approach in enumerate(["Approach2", 'Approach2t']):
        li.append([float((xr_combs.sel(Temperature = t,
                                       Region=r,
                                       Discount_factor=1.6,
                                       Historical_startyear=1990,
                                       Scenario='SSP2',
                                       Risk=0.5,
                                       Time=2030)[approach] - xr_dataread.GHG_ndc.sel(Region=r).mean(dim=['Conditionality', 'Ambition'])
                         ) / xr_dataread.GHG_ndc.sel(Region=r).mean(dim=['Conditionality', 'Ambition'])) for r in xr_dataread.Region])
    ndc_deviations.append(li)

In [44]:
#regs_onlyfew = np.array(['USA', 'CHN', 'TUR', 'AUS', 'IND', 'RUS', 'IDN', 'BRA', 'DEU', 'FRA', 'ESP', 'VEN', 'MEX', 'ZAF'])#np.array(xr_total.ISO.sel(ISO=all_countries_iso))[xr_total.Population.sel(Time=2020, ISO=all_countries_iso) > 5e6]

regs_onlyfew = ['EU', 'ARG', 'AUS', 'BRA', 'CAN', 'CHN', 'IND', 'IDN', 'JPN', 'MEX', 'RUS', 'SAU', 'ZAF', 'KOR', 'TUR', 'GBR', 'USA', 'ARE']
regs_onlyfew = np.array(regs_onlyfew)

ndc_deviations_onlyfew = []
for t_i, t in enumerate([1.6, 2.0]):
    li = []
    for a_i, approach in enumerate(["Approach2", 'Approach2t']):
        li.append([float((xr_combs.sel(Temperature = t,
                                       Region=r,
                                       Discount_factor=1.6,
                                       Historical_startyear=1990,
                                       Scenario='SSP2',
                                       Risk=0.5,
                                       Time=2030)[approach] - xr_dataread.GHG_ndc.sel(Region=r).mean(dim=['Conditionality', 'Ambition'])
                         ) / xr_dataread.GHG_ndc.sel(Region=r).mean(dim=['Conditionality', 'Ambition'])) for r in regs_onlyfew])
    ndc_deviations_onlyfew.append(li)

In [45]:
amount = len(regs_onlyfew)
sources = regs_onlyfew[np.argsort(ndc_deviations_onlyfew[0])[0][:amount]]
targets = regs_onlyfew[np.argsort(ndc_deviations_onlyfew[0])[1][:amount]]

targets_numeric = []
sources_numeric = []
for t_i in range(amount):
    s = sources[t_i]
    try:
        wh = np.where(targets == s)[0][0]
    except:
        wh = amount
    sources_numeric.append(t_i)
    targets_numeric.append(wh+amount)

for t_i in range(amount):
    t = targets[t_i]
    try:
        wh = np.where(sources == t)[0][0]
    except:
        sources_numeric.append(amount)
        targets_numeric.append(t_i)

## Plot

In [46]:
c_i = 0
fig = make_subplots(rows=2, cols=2, 
                                                    specs=[[{'type': 'choropleth'}, {'type': 'choropleth'}],
                                                           [{'type': 'sankey','colspan': 2}, {}]],
                                                    horizontal_spacing = 0, vertical_spacing=0.02)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
ts = ['Below 1.5 degrees', 'Well below 2 degrees', 'Below 3 degrees', 'Below 4 degrees']
colorbar_pc = dict(len=1, x=1)

for a_i, approach in enumerate(["A2_gdp_net", 'A2_trans']):
    if a_i == 0:
        cm= True
    else:
        cm= False
    #t[t >= 4] = np.nan
    fig.add_trace(
        go.Choropleth(
        locations=np.array(xr_dataread.Region),
        z = ndc_deviations[c_i][a_i],
        locationmode = 'ISO-3',
        colorscale = 'RdYlGn',
        zmax = 1,
        zmin = -1,
        text = [str(r)+": "+str(np.round(float(ndc_deviations[c_i][a_i][r_i])*100, 2))+"%" for r_i, r in enumerate(np.array(xr_dataread.Region))],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='white', 
        marker_line_width=0.3,
        colorbar=dict(len=1, x=1, tickvals = [-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1], ticktext = ["-100%", "-75", "-50%", "-25%", "0%", "+25%", "+50%", "+75%", "+100%"], title={"text":"Allocation<br>compared<br>to NDC"}),
        showscale=cm
    ), 1, a_i+1)
    fig.add_trace(
        go.Choropleth(
        locations=group_eu,
        z = np.array([ndc_deviations[c_i][a_i][np.where(all_regions_iso == "EU")[0][0]]]*len(group_eu)),
        locationmode = 'ISO-3',
        colorscale = 'RdYlGn',
        zmax = 1,
        zmin = -1,
        text = ["EU: "+str(np.round(float(ndc_deviations[c_i][a_i][np.where(all_regions_iso == "EU")[0][0]])*100, 2))+"%" for r_i, r in enumerate(np.array(xr_dataread.Region))],
        hovertemplate  = '%{text}',
        name="",
        marker_line_color='white', 
        marker_line_width=0.3,
        colorbar=dict(len=1, x=1, tickvals = [-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1], ticktext = ["-100%", "-75", "-50%", "-25%", "0%", "+25%", "+50%", "+75%", "+100%"], title={"text":"Allocation<br>compared<br>to NDC"}),
        showscale=cm
    ), 1, a_i+1)
fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

fig.add_trace(go.Sankey(
    arrangement='snap',
    node = dict(
      pad = 10,
      x = [0.05]*(amount)+[0.95]*(amount),
      y = list(np.linspace(0.001, 0.999, (amount)))+list(np.linspace(0.001, 0.999, (amount))),
    #   x = [0.001, 0.001, 0.001, 0.001, 0.999, 0.999, 0.999, 0.999],
    #   y = [0.001, 75/285, 160/285, 190/285, 0.001, 75/285, 130/285, 215/285], 
      thickness = 5,
      line = dict(color = "black", width = 0.5),
      label = ([all_regions_names[all_regions_iso == r][0]+": "+str(np.round(float(np.array(ndc_deviations_onlyfew[0][0])[regs_onlyfew == r])*100, 1))+"%" for r in regs_onlyfew[np.argsort(ndc_deviations_onlyfew[0])[0][:amount]]]+#+['Below top-'+str(amount)] +
              [all_regions_names[all_regions_iso == r][0]+": "+str(np.round(float(np.array(ndc_deviations_onlyfew[0][1])[regs_onlyfew == r])*100, 1))+"%" for r in regs_onlyfew[np.argsort(ndc_deviations_onlyfew[0])[1][:amount]]]),#+['Below top-'+str(amount)]),
      color = 'black'
    ),
    link = dict(
      source = sources_numeric, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = targets_numeric,#np.random.choice(np.arange(amount+1), size=amount+1, replace=False)+amount+1,
      value = [1]*len(targets_numeric),
      color = ['rgba(180,'+str(0+i/1.5)+','+str(0)+', 0.3)' for i in np.linspace(0, 255, len(sources_numeric))]
  )), 2, 1)

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)

fig.update_layout(
    font_size=15,
)
#fig.update_traces(showscale=False)
fig.add_annotation(xref='paper', yref='paper',ax=0, align='center', ay=-0, showarrow=False,
                    y=0.7,
                    x=1.025, 
                    textangle=90,
                    text="NDC ambitious enough <<<",
                    font=dict(color='black', size=16),
                )
fig.add_annotation(xref='paper', yref='paper',ax=0, align='center', ay=-0, showarrow=False,
                    y=0.2,
                    x=1.025, 
                    textangle=90,
                    text=">>> NDC not ambitious enough",
                    font=dict(color='black', size=16),
                )

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False,
                    y=1.0,
                    x=0.07,
                    text="<b>(a) Allocation-NDC comparison for approach 2</b>",
                    font=dict(color='black', size=16),
                )
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False,
                    y=1.0,
                    x=0.86,
                    text="<b>(b) Allocation-NDC comparison for approach 2<br>(transitioning from current emissions in 20 years)</b>",
                    font=dict(color='black', size=16),
                )
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False,
                    y=0.52,
                    x=0.04,
                    text="<b>(c) Ordering of G20 and United Arab Emirates</b>",
                    font=dict(color='black', size=16),
                )

fig.update_layout(height=1200, width=1750)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1112\3764947325.py:62: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_1112\3764947325.py:63: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [67]:
fig.write_image("K:/Code/EffortSharing/Figures/Paper_Robiou2023/Figure_3.png", scale=2)